*bucket2 file takes train/test/val into account*

In [1]:
import os
from google.cloud import storage
from PIL import Image
from io import BytesIO
import pandas as pd

In [2]:
client = storage.Client(project='prime-basis-374911')
bucket = client.get_bucket('kitchen-ai-images')

#blob = bucket.get_blob('BROCCOLI_0.jpg')
#img = Image.open(BytesIO(blob.download_as_bytes()))
# img

In [4]:
path_to_dataset = os.path.join("..", "data")

In [5]:
#ideas include updating to allow for getting specific class
#or stopping at specific class
#or loading n of each class
#set classes to ignore

##SKIPPED MUSHROOM 21
#filenames_to_skip = ["mushroom_21.jpg"]

#set to empty list just after, used for jumping in after image error
classes_to_skip = [
'asparagus',
'beetroot',
'broccoli',
'cabbage',
'capsicum',
'carrot',
'cauliflower',
'celery',
'corn',
'cucumber',
'eggplant',
'garlic',
'ginger',
'leek',
'lettuce']
classes_to_skip = []



mini_set_stop_class = "capsicum"

#used to correctly index incoming data from 2 sets
multi_set_input_class = ["spinach"]

In [6]:
def save_bucket_data_to(miniset=False):
    all_files = bucket.list_blobs()
    
    #set path to dataset
    # if miniset:
    #     path_to_dataset = os.path.join("..", "test_data", "0_test_miniset")
    # else:
    #     path_to_dataset = os.path.join("..", "data")
    
    #path_to_dataset = os.path.join("..", "test_data", "1_test_split_set", "test")
    
    path_to_data_folder = os.path.join("..", "test_data", "3_test_from_bucket2")
    
    #class_limit = 50
    class_count = 0
    train_count = 0
    test_count = 0
    val_count = 0
    
    current_class = ""
    #this_file = next(all_files)
    #for i in range(10):
    
    for this_file in all_files:
        und_split = this_file.name.split('_')
        dot_split = this_file.name.split('.')
        food_bucket_path = this_file.path #to track old names

        #get info from bucket path name
        food_class = und_split[0].lower()
        file_ext = dot_split[1] #print(img_ext)
        file_num = dot_split[0].split('_')[-1] #print(file_num)
        
        #data_split = "train" "test" "val"
        data_split = dot_split[0].split('_')[-2]      
        
        #create path to correct folder
        path_to_dataset = os.path.join(path_to_data_folder, data_split)
        
        #create new file path
        new_filename = food_class + "_" + file_num  + ".jpg" #"." + file_ext
        new_dir_path = os.path.join(path_to_dataset, food_class) #used to create folder if not already existing
        new_path = os.path.join(new_dir_path,new_filename)   #class/class-n.ext
        
        #tracking
        if food_class != current_class:
            #if statement avoids printing empty for first
            if current_class:
                print(f"loaded total {class_count} of {current_class}")
                print(f"{train_count} train")
                print(f"{val_count} val")
                print(f"{test_count} test")
                    
            current_class = food_class
            class_count = 0
            train_count, test_count, val_count = 0,0,0
        
        ## STOPPING / SKIPPING CONDITIONS
        
        #STOP AT GARLIC FOR MINISET
        #if miniset and (food_class == mini_set_stop_class):
        #    break
        
        if food_class in classes_to_skip:
            continue
        #if new_filename in filenames_to_skip:
        #    continue
        #if class_count > class_limit:
        #    continue
            
        try:
            #get image from bucket
            file_image = Image.open(BytesIO(this_file.download_as_bytes()))
            #convert to rgb ==> jpg
            rgb_image = file_image.convert('RGB')

            #save image
            os.makedirs(new_dir_path, exist_ok=True) #True means don't create new dir if already exists #os.makedirs(path, exist_ok=True)
            test_img = rgb_image.save(new_path)
            
            class_count += 1
            
            if data_split == "train":
                train_count += 1
            if data_split == "test":
                test_count += 1
            if data_split == "validation":
                val_count += 1
            
        except:
            print(f"error loading from {food_bucket_path} \n to {new_path}")
    
    print(f"Done! loaded {len(all_files)}")

In [12]:
#save_bucket_data_to()

In [7]:
save_bucket_data_to()

loaded 0 of 
loaded 1400 of broccoli
loaded 1400 of cabbage
loaded 1400 of carrot
loaded 1400 of cauliflower
loaded 1400 of cucumber
loaded 1400 of potato
loaded 1400 of pumpkin
